In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\UPSIT
data = pd.read_csv('UPSIT3Categ_Methylome_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\UPSIT


,Sentrix,PATNO,HYS,Gender,APPRDX,UPSIT_3,UPSIT_Total,cg06067658,cg15442538,cg25928446,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200991620021_R01C01,3001,2,1,1,1,25,0.088679,0.890148,0.719238,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608


In [3]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','UPSIT_3','UPSIT_Total'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7258064516129032

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100


BorutaPy finished running.

Iteration: 	10 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	5672


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1F2EB2C3B40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1F2EB2C3B40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
0,APPRDX,1
4055,cg24628866,2
2852,cg14398353,3


In [10]:
Top50.to_csv ('UPSIT_BorutaMale_Top50.csv', index = False)
selected_rf_features.to_csv ('UPSIT_BorutaMale_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
